In [3]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.8.0-cp314-cp314-win_amd64.whl.metadata (11 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.8.0-cp314-cp314-win_amd64.whl (8.1 MB)
   ---------------------------------------- 0.0/37.3 MB ? eta -:--:--
   - -------------------------------------- 1.3/37.3 MB 6.5 MB/s eta 0:00:06
   -- ------------------------------------- 2.1/37.3 MB 4.9 MB/s eta 0:00:08
   -- ------------------------------------- 2.4/37.3 MB 3.7 MB/s eta 0:00:10
   --- ------------------------------------ 2.9/37.3 MB 3.3 MB/s eta 0:00:11
   --- ------------------------------------ 3.4/37.3 MB 3.2 MB/s eta 0:00:11
   ---- ----------------------------------- 3.9/37.3 MB 3.2 MB/s eta 0:00:11
   ---- ----------------------------------- 4.2/37.3 MB 2.9 MB/s eta 0:00:12
   ---- ----------------------------------- 4.5/37.3 MB 2.8 MB/s eta 0:00:12
   ----- --------------

In [1]:
import numpy as np
import pandas as pd
import os
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
path='C:\\Users\\User\\Documents\\Zomato Restaurant Project'
dataset=pd.read_csv(os.path.join(path,'zomato.csv'),encoding='latin-1')

In [33]:
dataset.info()

<class 'pandas.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   str    
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   str    
 4   Address               9551 non-null   str    
 5   Locality              9551 non-null   str    
 6   Locality Verbose      9551 non-null   str    
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   str    
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   str    
 12  Has Table booking     9551 non-null   str    
 13  Has Online delivery   9551 non-null   str    
 14  Is delivering now     9551 non-null   str    
 15  Switch to order menu  9551 non-n

In [34]:
country_df=pd.read_excel(os.path.join(path,'Country-Code.xlsx'))

In [35]:
final_dataset=dataset.merge(country_df,on='Country Code',how='left')
columns=['Country Code','Locality Verbose','Is delivering now','Switch to order menu','Rating color','Currency']
final_dataset.drop(columns=columns,inplace=True)
final_dataset=final_dataset[final_dataset['Country']=='India']

In [36]:
final_dataset.columns

Index(['Restaurant ID', 'Restaurant Name', 'City', 'Address', 'Locality',
       'Longitude', 'Latitude', 'Cuisines', 'Average Cost for two',
       'Has Table booking', 'Has Online delivery', 'Price range',
       'Aggregate rating', 'Rating text', 'Votes', 'Country'],
      dtype='str')

In [37]:
final_dataset.head()

,Restaurant ID,Restaurant Name,City,Address,Locality,Longitude,Latitude,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Price range,Aggregate rating,Rating text,Votes,Country
624,3400025,Jahanpanah,Agra,"E 23, Shopping Arcade, Sadar Bazaar, Agra Cant...",Agra Cantt,78.011544,27.161661,"North Indian, Mughlai",850,No,No,3,3.9,Good,140,India
625,3400341,Rangrezz Restaurant,Agra,"E-20, Shopping Arcade, Sadar Bazaar, Agra Cant...",Agra Cantt,0.000000,0.000000,"North Indian, Mughlai",700,No,No,2,3.5,Good,71,India
626,3400005,Time2Eat - Mama Chicken,Agra,"Main Market, Sadar Bazaar, Agra Cantt, Agra",Agra Cantt,78.011608,27.160832,North Indian,500,No,No,2,3.6,Good,94,India
627,3400021,Chokho Jeeman Marwari Jain Bhojanalya,Agra,"1/48, Delhi Gate, Station Road, Raja Mandi, Ci...",Civil Lines,77.998092,27.195928,Rajasthani,400,No,No,2,4.0,Very Good,87,India
628,3400017,Pinch Of Spice,Agra,"23/453, Opposite Sanjay Cinema, Wazipura Road,...",Civil Lines,78.007553,27.201725,"North Indian, Chinese, Mughlai",1000,No,No,3,4.2,Very Good,177,India


In [43]:
final_dataset['Cuisines']=final_dataset['Cuisines'].str.replace(','," ")

In [60]:
final_dataset.sample(2)

,Restaurant ID,Restaurant Name,City,Address,Locality,Longitude,Latitude,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Price range,Aggregate rating,Rating text,Votes,Country
4192,4019,Meenu Caterers,New Delhi,"C4F, DDA Market, Janakpuri, New Delhi",Janakpuri,77.090369,28.616886,North Indian Chinese,600,No,Yes,2,2.7,Average,82,India
5544,18471517,Taaj Kitchen,New Delhi,"69/6-A, Najafgarh Road, Opposite Moments Mall,...",Moti Nagar,0.000000,0.000000,Asian North Indian Mughlai,700,Yes,No,2,0.0,Not rated,1,India


<h1> Cases:</h1>
<ol>
    <li>show top 10 similar restaurants to the one user selected based on cuisines filter by city.
    <li>conditions:locality is given by user,top 10 similar restaurants in locality by cuisines.
    <li>conditions: minimum rating given
        <ul>
            <li>price range is not given-->sort them from primium to cheap</li>
            <li>price range is given-->filter by price range and then sort them in desc</li>
        </ul></li>
    <li>conditions: when ratings,price range,table booking,online delivery -->re rank them based on condition</li>
</ol>

In [25]:
tfidf= TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2),
    lowercase=True
)
tfidf_matrix=tfidf.fit_transform(final_dataset['Cuisines'])
similarity_matrix=cosine_similarity(tfidf_matrix,tfidf_matrix)

In [51]:
restaurant_index=pd.Series(final_dataset.index,index=final_dataset['Restaurant Name']).drop_duplicates()

In [53]:
restaurant_index

Restaurant Name
Jahanpanah                                624
Rangrezz Restaurant                       625
Time2Eat - Mama Chicken                   626
Chokho Jeeman Marwari Jain Bhojanalya     627
Pinch Of Spice                            628
                                         ... 
D Cabana                                 9271
Kaloreez                                 9272
Plot 17                                  9273
Vista - The Park                         9274
Flying Spaghetti Monster                 9275
Length: 8652, dtype: int64

In [61]:
restaurant_name='Taaj Kitchen'
idx=restaurant_index[restaurant_name]
scores=list(enumerate(similarity_matrix[idx]))
sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
sim_scores=sorted_scores[1:11]
scores

[(0, np.float64(0.2712839085765052)),
 (1, np.float64(0.2712839085765052)),
 (2, np.float64(0.5062891110704909)),
 (3, np.float64(0.0)),
 (4, np.float64(0.2936764312220811)),
 (5, np.float64(0.1703296988368838)),
 (6, np.float64(0.2712839085765052)),
 (7, np.float64(0.0)),
 (8, np.float64(0.0)),
 (9, np.float64(0.2712839085765052)),
 (10, np.float64(0.2965575372814151)),
 (11, np.float64(0.04819857690263123)),
 (12, np.float64(0.1984092309362678)),
 (13, np.float64(0.25881664419522177)),
 (14, np.float64(0.2965575372814151)),
 (15, np.float64(0.10031513971196292)),
 (16, np.float64(0.2712839085765052)),
 (17, np.float64(0.0)),
 (18, np.float64(0.20051827126927751)),
 (19, np.float64(0.22344713516746623)),
 (20, np.float64(0.11647401447249238)),
 (21, np.float64(0.66770641794534)),
 (22, np.float64(0.0)),
 (23, np.float64(0.1140596268051269)),
 (24, np.float64(0.0)),
 (25, np.float64(0.0)),
 (26, np.float64(0.0)),
 (27, np.float64(0.09177613747698457)),
 (28, np.float64(0.14163863576325

In [62]:
restaurant_indices=[i[0] for i in sim_scores]
recommend=final_dataset.iloc[restaurant_indices].copy()
recommend['similarity_score']=[i[1] for i in sim_scores]
recommend

,Restaurant ID,Restaurant Name,City,Address,Locality,Longitude,Latitude,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Price range,Aggregate rating,Rating text,Votes,Country,similarity_score
971,18466420,Chaudhary Chaap & Chinese,Faridabad,"Shop 137, Huda Market, Sector 16, Faridabad",Sector 16,77.318702,28.410665,Chinese North Indian,300,No,No,1,0.0,Not rated,0,India,1.0
978,304001,Eat N Treat,Faridabad,"145, Main Market, Sector 17, Faridabad",Sector 17,77.326602,28.410102,Chinese North Indian,400,No,No,1,2.8,Average,20,India,1.0
986,304004,Tmos Moving Feast,Faridabad,"156, Main Market, Sector 17, Faridabad",Sector 17,77.326097,28.410116,Chinese North Indian,700,No,Yes,2,3.3,Average,76,India,1.0
1062,18292444,Touch of Spice,Faridabad,"B-195, City Square Complex, Green Field Colony...",Sector 43,77.297384,28.462107,Chinese North Indian,600,No,No,2,0.0,Not rated,0,India,1.0
1306,18034048,Night Food Xprs,Gurgaon,"DLF Phase 1, Gurgaon",DLF Phase 1,77.099388,28.469341,Chinese North Indian,650,No,No,2,3.1,Average,12,India,1.0
1331,18417487,Prabhat Fast Food Corner,Gurgaon,"Opposite Plot 14, Sector 18, Near DLF Phase 2,...",DLF Phase 2,77.084372,28.490311,Chinese North Indian,200,No,No,1,0.0,Not rated,1,India,1.0
1354,18492107,Foodies Nation,Gurgaon,"U-10/25, DLF phase 3, Gurgaon, Haryana-122001,...",DLF Phase 3,0.000000,0.000000,Chinese North Indian,300,No,No,1,3.4,Average,19,India,1.0
1363,1694,Nikhil Food Point,Gurgaon,"U-69/3, DLF Phase 3, Gurgaon",DLF Phase 3,77.094764,28.489970,Chinese North Indian,200,No,No,1,2.7,Average,19,India,1.0
1472,312111,Jaguar,Gurgaon,"Opposite Genpact, Golf Course Road, Gurgaon",Golf Course Road,77.098634,28.449925,Chinese North Indian,800,Yes,No,2,2.8,Average,14,India,1.0
1665,18324432,Chaska Food Corner,Gurgaon,"Shop 1, Bestech Park View Residency Market, Se...",Palam Vihar,77.022532,28.497571,Chinese North Indian,700,No,No,2,0.0,Not rated,1,India,1.0
